In [133]:
import numpy as np
import scipy as scp
from random import seed
from random import random
from random import randrange
from random import uniform

In [167]:
class BaseNode:
    def __init__(self, function):
        self.function = function
        self.bias = random()
    def mutateBias(self):
        self.bias *= uniform(0, 2)

class StartNode(BaseNode):
    def __init__(self, function):
        super().__init__(function)
        self.values = []
        self.weights = []
    def mutate(self, probability):
        if random() < probability:
            print('mutating!')
            num = randrange(len(self.weights))
            if num == len(self.weights):
                self.mutateBias()
            else:
                self.weights[num] *= uniform(0, 2)
    def setVal(self, val):
        self.values = val
        self.weights = []
        for i in range(0, len(val)):
            self.weights.append(uniform(-1,1))
    def getResult(self):
        if not self.values:
            raise Exception('No values')
        else:
            sum = self.bias
            for i in range(len(self.values)):
                sum = self.weights[i]*self.values[i]
            return self.function(sum)

class Node(BaseNode):
    def __init__(self, function):
        super().__init__(function)
        self.prevNodes = []
        self.weights = []
    def mutate(self, probability):
        if random() < probability:
            print('mutating!')
            num = randrange(len(self.weights))
            if num == len(self.weights):
                self.mutateBias()
            elif num < len(self.weights):
                self.weights[num] *= uniform(0, 2)
        for n in self.prevNodes:
            n.mutate(probability)
    def addPrevNode(self, node):
        self.prevNodes.append(node)
        self.weights.append(uniform(-1, 1))
    def addPrevNodes(self, nodes):
        for n in nodes:
            self.addPrevNode(n)
    def getResult(self):
        if not self.prevNodes:
            raise Exception('No previous nodes')
        else:
            sum = self.bias
            for i in range(len(self.weights)):
                sum += self.weights[i]*self.prevNodes[i].getResult()
            return self.function(sum)

class NeuralNetwork:
    def __init__(self, shape, function):
        self.startNodes = []
        if not shape:
            self.startNodes.append(StartNode(function))
            self.endNode = self.startNodes[0]
        else:
            for i in range(shape[0]):
                self.startNodes.append(StartNode(function))
            shape.remove(shape[0])
            prev = self.startNodes
            for n in shape:
                if n < 1:
                    raise Exception('No layer can have less than one node')
                tmp = []
                for i in range(n):
                    tmp.append(Node(function))
                    tmp[i].addPrevNodes(prev)
                prev = tmp
            self.endNode = Node(function)
            self.endNode.addPrevNodes(prev)
    def getResult(self):
        return self.endNode.getResult()
    def setVal(self, value):
        for n in self.startNodes:
            n.setVal(value)
    def mutate(self, probability, mutProb):
        if random() < probability:
            self.endNode.mutate(mutProb)

In [169]:
wat = NeuralNetwork([2, 3, 2], lambda x: x)
wat.setVal([4, 6, 7])
print(wat.getResult())
wat.mutate(1, 0.4)

1.9596642689866535
mutating!
mutating!
mutating!
mutating!
mutating!
mutating!
mutating!
mutating!
mutating!


[4, 4]


AttributeError: 'list' object has no attribute 'size'